In [ ]:
#graph with the subplots for each year
# mobility_trips_years = mobility_trips_motive[years]

# # Melt the data to create a 'year' column
# mobility_trips_years = mobility_trips_years.reset_index().melt(id_vars=['motive'], var_name='year', value_name='trips')

# fig = px.bar(mobility_trips_years, x='trips', y='motive', color='motive', facet_col='year', orientation='h', hover_name='trips', width=1100, height=400)

# fig.update_layout(title={
#     'text': 'Average amount of trips for every vehicle combined per day from 2018 to 2022',
#     'x': 0.5
# })
# fig.update_yaxes(title_text="Travel motives")
# fig.show()

In [ ]:
# Reshape data for year to year plot
fuel_prices_yearly = fuel_prices.resample('Y').mean() 
fuel_prices_yearly['Average'] = fuel_prices_yearly.mean(axis=1)
fuel_prices_yearly['Year'] = fuel_prices_yearly.index.strftime('%Y')
fuel_prices_yearly = fuel_prices_yearly.set_index('Year')
fuel_prices_yearly.reset_index(level=0, inplace=True)

# Calculate Year-over-Year percentage changes for each fuel type
fuel_data_yearly = fuel_prices_yearly
fuel_types = ['Euro95', 'Diesel', 'LPG', 'Average']
for fuel_type in fuel_types:
    column_name = f"{fuel_type}_YoY"
    fuel_data_yearly[column_name] = fuel_data_yearly[fuel_type].pct_change() * 100

# Insert '0' YoY change for the year 2019 (second year)
for fuel_type in fuel_types:
    column_name = f"{fuel_type}_YoY"
    fuel_data_yearly.at[0, column_name] = 0

# Create a long-form DataFrame for Plotly Express
melted_fuel_data = fuel_data_yearly.melt(id_vars='Year', value_vars=[f"{fuel_type}_YoY" for fuel_type in fuel_types], var_name='Fuel_Type', value_name='YoY_Change')
Fuel_Yoy = pivot_df = melted_fuel_data.pivot(index='Year', columns='Fuel_Type', values='YoY_Change').reset_index()

display(Fuel_Yoy)

In [ ]:
# Plot the Year-over-Year changes using Plotly Express
fig = px.line(melted_fuel_data, x='Year', y='YoY_Change', color='Fuel_Type', title='Year-over-Year Fuel Price Changes')
fig.update_layout(xaxis_title='Year', yaxis_title='YoY Change (%)', legend_title='Fuel Type', width=1100, height=400)
fig.add_vrect(x0=2.25, x1=4, 
              annotation_text="COVID ", annotation_position="top left",
              fillcolor="green", opacity=0.25, line_width=0)
fig.show()

In [ ]:
yoy_changes = mobility_trend_dist.copy()

# Calculate year-over-year changes for each mode while setting the first year's change to 0
yoy_changes.iloc[:, 1:] = yoy_changes.iloc[:, 1:].pct_change(axis=1) * 100
yoy_changes.iloc[:, 1] = 0  # Set the first year's change to 0 for all modes

yoy_changes

In [ ]:
# Plotting the year-over-year changes for each mode
melted_yoy = yoy_changes.melt(id_vars='modes', var_name='Year', value_name='YoY Change %')
fig = px.line(melted_yoy, x='Year', y='YoY Change %', color='modes',
    title='Year-over-Year Changes for distance travelled in different Modes of Transportation')
fig.update_layout(height=600, width=800 )
fig.show()